Hier schmeißen wir die komplette spacy library auf unsere Sätze und werten einfach mal alles aus. 


In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 205, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 342, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    collected.requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 481, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/pyth

In [ ]:
# show cpu specs 
# !cat /proc/cpuinfo
# !cat /proc/meminfo

In [2]:
import spacy
from spacy.lang.en import English
from spacy.lang.es import Spanish
from spacy import tokenizer
from spacy.tokens import DocBin

In [3]:
import pandas as pd
import numpy as np
import time
import glob

In [4]:
# other imports
import os
from os import path
from google.colab import drive

In [5]:
# mount google drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
nlp_en = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")

In [ ]:
# process more shit in parallel 
def read_data(inputfile):
    lines = load_data(inputfile)
    df = pd.DataFrame(lines, columns=['content'])
    return df


def tag_dep_pipe(doc):
    tag_dep_list = [[tok.text, tok.lemma_, tok.pos_, tok.tag_, tok.dep_] for tok in doc
                  if tok.is_alpha and not tok.is_stop]
    return tag_dep_list


def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp_en.pipe(texts, batch_size=2000):
        preproc_pipe.append(tag_dep_pipe(doc))
    return preproc_pipe


def chunker(iterable, total_length, chunksize):
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))


def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]


def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp_en.pipe(texts, batch_size=2000):
        preproc_pipe.append(tag_dep_pipe(doc))
    return preproc_pipe


def preprocess_parallel(texts, chunksize=100):
    executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
    do = delayed(process_chunk)
    tasks = (do(chunk) for chunk in chunker(texts, len(df_preproc), chunksize=chunksize))
    result = executor(tasks)
    return flatten(result)


df_preproc['preproc_parallel'] = preprocess_parallel(df_preproc['clean'], chunksize=10000)

In [ ]:
# using spacy DocBin 
doc_bin_en = DocBin()
trl_data = load_data(translated_en_path)
print(trl_data[0])
for i, doc in enumerate(nlp_en.pipe(" ".join(trl_data), batch_size=2000)):
  print("Current batch is ", i)
  doc_bin_en.add(doc)
bytes_data = doc_bin_en.to_bytes()

In [ ]:
with open(path.join(eval_path, "doc_bin_container_en"), 'wb') as f:
  f.write(bytes_data)

In [ ]:
def pipe_en_translations():
  doc_bin_en = DocBin()
  trl_data = load_data(translated_en_path)
  for number, data in enumerate():
    for i, doc in enumerate(nlp_en.pipe(" ".join(trl_data), batch_size=5000)):
      print("Current batch is ", i)
      doc_bin_en.add(doc)
    bytes_data = doc_bin_en.to_bytes()
    with open(path.join(eval_path, f'doc_bin_container_en_{number}'), 'wb') as f:
      f.write(bytes_data)


def pipe_es_translations(translations):
  doc_bin_en = DocBin()
  trl_data = load_data(translated_en_path)
  for number, data in enumerate():
    for i, doc in enumerate(nlp_es.pipe(" ".join(trl_data), batch_size=5000)):
      print("Current batch is ", i)
      doc_bin_en.add(doc)
    bytes_data = doc_bin_en.to_bytes()
    with open(path.join(eval_path, f'doc_bin_container_es_{number}'), 'wb') as f:
      f.write(bytes_data)

In [ ]:
# deserialize later
doc_bin_path = path.join(eval_path, "doc_bin_container_en")
doc_bin_deserialized = DocBin().from_bytes(doc_bin_path)
docs = list(doc_bin_deserialized.get_docs(nlp_en.vocab))

In [7]:
# load data function
def load_data(file_path):
  try:
    with open(file_path, "r") as f:
      lines = f.readlines()
      return [s.replace("\n", "") for s in lines]
  except EnvironmentError as e:
      print("Could not read file because exception occured: ", e.with_traceback())

In [8]:
# divide data into batches
def create_batches(data, batchsize):
  """Divides a batch of data into batches of given size.

  Args:
    data: The data to split in list format.
    batchsize: Size of desired batches.

  Returns:
    Iterable of all batches with batchsize.
  
  Raises:
    AssertionError if the size of the data is not a multiple of batchsize.

  """
  assert len(data) % batchsize == 0
  batches = []
  num_batches = len(data) // batchsize
  for i in range(num_batches):
    batches.append(data[i*batchsize:(i+1)*batchsize])
  return batches

In [9]:
# data paths
translated_path = "/content/drive/MyDrive/mlt_final_project/translated"
preprocessed_path = "/content/drive/MyDrive/mlt_final_project/preprocessed"
it_trl_path = "/content/drive/MyDrive/mlt_final_project/iterative_translation"
eval_path = path.join("/content/drive/MyDrive/mlt_final_project", "evaluation")
translated_en_path = path.join(translated_path, "de-en", "translated.en")
translated_es_path = path.join(translated_path, "de-es", "translated.es")
reference_en_path = path.join(preprocessed_path, "de-en", "pp_deen.en")
reference_es_path = path.join(preprocessed_path, "de-es", "pp_dees.es")
# translation files from english to german
it_trl_ende_paths = [path.join(it_trl_path, f'backward_trl_ende_{num}.txt') for num in range(5)]
# translation files from german to english
it_trl_deen_paths = [path.join(it_trl_path, f'forward_trl_deen_{num}.txt') for num in range(5)]
# translation files from spanish to german
it_trl_esde_paths = [path.join(it_trl_path, f'backward_trl_esde_{num}.txt') for num in range(5)]
# translation files from german to spanish
it_trl_dees_paths = [path.join(it_trl_path, f'forward_trl_dees_{num}.txt') for num in range(5)]

In [ ]:
# load data
trl_data = load_data(translated_en_path) #, translated_es_path

In [ ]:
# evaluation
trl_en_batches = create_batches(trl_data[0], 1000)
trl_es_batches = create_batches(trl_data[1], 1000)
# len(trl_en_batches)

In [ ]:
# for batch in trl_en_batches:
#   trl_doc_en = nlp_en(" ".join(trl_data[0]))
#   trl_doc_es = nlp_es(" ".join(trl_data[1]))
trl_doc_en = nlp_en(" ".join(trl_data[0][:2]))

<class 'spacy.tokens.doc.Doc'>


In [22]:
def token_extraction(spacy_nlp_obj):
  tokens = []
  for token in spacy_nlp_obj:
    tokens.append([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop])
  return tokens

def create_token_dataframes(trl_batch, language, name):
  # test_path = path.join("/content/drive/MyDrive/mlt_final_project", "test")
  dataframes = []
  for number, batch in enumerate(trl_batch):
    start = time.time()
    print(f'Processing batch number {number+1} of {len(trl_batch)}...')
    if language == "en":
      trl_nlp_obj = nlp_en(" ".join(batch))
      # trl_nlp_obj.to_disk(path.join(spacy_models_path, "en", f'en_spacy_model_{name}_{number}'))
    else:
      trl_nlp_obj = nlp_es(" ".join(batch))
      # trl_nlp_obj.to_disk(path.join(spacy_models_path, "es", f'es_spacy_model_{name}_{number}'))
    tokens = token_extraction(trl_nlp_obj)
    df = pd.DataFrame(tokens, columns=['Text', 'Lemma', 'Pos', 'Tag', 'Dep', 'Shape', 'Alpha', 'Stop'])
    # df.to_csv(f'{test_path}/{name}_{number}.csv')
    dataframes.append(df)
    end = time.time()
    print(f'Processing batch number {number+1} took {end-start} seconds.')
  return dataframes

In [10]:
def concatenate_df_batches(df_batches: list):
  return pd.concat(df_batches, ignore_index=True)

In [11]:
# ['Text', 'Lemma', 'Pos', 'Tag', 'Dep', 'Shape', 'Alpha', 'Stop']
 
# count hapax legomena, bi legomena tri legomena
def count_legomas(df):
  words, word_counts = np.unique(df['Text'].to_numpy(), return_counts=True)
  hapax_legoma = words[word_counts == 1]
  bi_legoma = words[word_counts == 2]
  tri_legoma = words[word_counts == 3]
  return hapax_legoma.size, bi_legoma.size, tri_legoma.size


# filters stop characters and non alphanumericals from df
def filter_stop_non_alpha(df):
  filtered_df = df.loc[(df['Stop'] == False) & (df['Alpha'] == True)]
  return filtered_df.drop(columns=['Stop', 'Alpha'])

In [12]:
# shannon entropy
def shannon_entropy_o(dataframe):
  lemma_frames = dataframe.groupby('Lemma')
  entropy_values=[]
  for lemma, frame in lemma_frames:
    words, word_counts = np.unique(frame['Text'].to_numpy(), return_counts=True)
    normalized_word_counts = word_counts / np.sum(word_counts)
    entropy = -np.sum(normalized_word_counts * np.log(normalized_word_counts))
    entropy_values.append(entropy)
  return np.mean(entropy_values)


# Simpson's diversity
def simpson_diversity_o(dataframe):
  lemma_frames = dataframe.groupby('Lemma')
  diversity_values=[]
  for lemma, frame in lemma_frames:
    words, word_counts = np.unique(frame['Text'].to_numpy(), return_counts=True)
    normalized_word_counts = word_counts / np.sum(word_counts)
    diversity = 1/np.sum(normalized_word_counts**2)
    diversity_values.append(diversity)
  return np.mean(diversity_values)

In [13]:
# Lexical Frequency Profile
def get_lfp_o(data_frame, return_index_dict=False):
  words, word_counts = np.unique(data_frame['Text'].to_numpy(), return_counts=True)
  sorted_indices = np.argsort(word_counts)
  # potential to get the actual most used words
  sorted_words = words[sorted_indices]
  sorted_word_counts = word_counts[sorted_indices]
  denominator = np.sum(sorted_word_counts)
  first_thousand_percentage = np.sum(sorted_word_counts[:1000])/denominator
  second_thousand_percentage = np.sum(sorted_word_counts[1000:2000])/denominator
  rest_percentage = np.sum(sorted_word_counts[2000:])/denominator
  index_dict = None
  if return_index_dict:
    for word, index in zip(sorted_words, sorted_indices):
      index_dict[word] = index
  return [first_thousand_percentage, second_thousand_percentage, rest_percentage], index_dict

In [31]:
# nici functions
def ttr_ratio(dataframe, lemma=False, sort_punctuation_out=True, sort_stopwords_out=False):
  if sort_stopwords_out:
      dataframe = dataframe.loc[dataframe['Stop'] == False]
  if sort_punctuation_out:
      dataframe = dataframe.loc[dataframe['Tag'] != 'PUNCT']
  if lemma:
    words = dataframe["Lemma"].to_numpy()
  else:
    words = dataframe["Text"].to_numpy()
  return len(np.unique(words))/len(words)


def shannon_entropy(dataframe, normalizing_constant=None):
  if normalizing_constant is None:
    normalizing_constant = 0
    count=True
  else:
    count=False
  if 'relevant_lemma' in dataframe.columns:
    subframe = dataframe.loc[dataframe['relevant_lemma'] == True]
  else:
    subframe = dataframe
  lemma_frames = subframe.groupby('Lemma')
  entropy_values=[]
  for lemma, frame in lemma_frames:
    #print(frame)
    #print(".")
    words, word_counts = np.unique(frame['Text'].to_numpy(), return_counts=True)
    # TODO: decide whether we want to remove all words that have in theory more 
    # than one lemma but not in our data
    if len(words) < 2:
      continue
    if count:
      normalizing_constant += 1
    normalized_word_counts = word_counts / np.sum(word_counts)
    entropy = -np.sum(normalized_word_counts * np.log(normalized_word_counts))
    entropy_values.append(entropy) 
  return np.sum(entropy_values)/normalizing_constant, normalizing_constant


def simpson_diversity(dataframe):
  if 'relevant_lemma' in dataframe.columns:
    subframe = dataframe.loc[dataframe['relevant_lemma'] == True]
  else:
    subframe = dataframe
  lemma_frames = subframe.groupby('Lemma')
  diversity_values=[]
  for lemma, frame in lemma_frames:
    if len(frame) == 1:
      continue
    words, word_counts = np.unique(frame['Text'].to_numpy(), return_counts=True)
    # TODO: decide whether we want to remove all words that have in theory more 
    # than one lemma but not in our data
    if len(words) < 2:
      continue
    normalized_word_counts = word_counts / np.sum(word_counts)
    diversity = np.sum(normalized_word_counts**2)
    diversity_values.append(diversity)
  return np.mean(diversity_values)


def get_lfp(data_frame, index_dict=None, return_index_dict=False, sort_punctuation_out=True, sort_stopwords_out=False):
    if sort_stopwords_out:
      data_frame = data_frame.loc[data_frame['Stop'] == False]
    if sort_punctuation_out:
      data_frame = data_frame.loc[data_frame['Tag'] != 'PUNCT']

    words, word_counts = np.unique(data_frame['Lemma'].to_numpy(), return_counts=True)
    if index_dict is not None:
      new_word_indices = []
      restructured_indices = []
      for word in words:
        if word in index_dict.keys():
          restructured_indices.append(index_dict[word])
          new_word_indices.append(False)
        else:
          new_word_indices.append(True)
      sorted_word_counts = word_counts[restructured_indices]          
      new_words = words[new_word_indices]
      new_word_counts = word_counts[new_word_indices]
      sorted_word_counts = np.append(sorted_word_counts, new_word_counts)
      denominator = np.sum(sorted_word_counts)
      first_thousand_percentage = np.sum(sorted_word_counts[:1000])/denominator
      second_thousand_percentage = np.sum(sorted_word_counts[1000:2000])/denominator
      rest_percentage = np.sum(sorted_word_counts[2000:])/denominator
      new_word_percentage = np.sum(new_word_counts)/denominator
      index_dict = None
      return [first_thousand_percentage, second_thousand_percentage, rest_percentage], index_dict  
    else:
      sorted_indices = np.argsort(word_counts)[::-1]
      # potential to get the actual most used words
      sorted_words = words[sorted_indices]
      sorted_word_counts = word_counts[sorted_indices]
      denominator = np.sum(sorted_word_counts)
      #print(sorted_word_counts)
      first_thousand_percentage = np.sum(sorted_word_counts[:1000])/denominator
      second_thousand_percentage = np.sum(sorted_word_counts[1000:2000])/denominator
      rest_percentage = np.sum(sorted_word_counts[2000:])/denominator
      index_dict = None
      if return_index_dict:
        index_dict = {}
        for  index, word in enumerate(sorted_words[:2000]):
          index_dict[word] = index
      return [first_thousand_percentage, second_thousand_percentage, rest_percentage], index_dict

In [29]:
dfs_path = path.join("/content/drive/MyDrive/mlt_final_project", "dfs")
paths = glob.glob(f'{dfs_path}/*')
paths

['/content/drive/MyDrive/mlt_final_project/dfs/pp_deen.en',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_deen_1',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_deen_0',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_deen_2',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_deen_3',
 '/content/drive/MyDrive/mlt_final_project/dfs/translated.en',
 '/content/drive/MyDrive/mlt_final_project/dfs/pp_dees.es',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_1',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_0',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_2',
 '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_3',
 '/content/drive/MyDrive/mlt_final_project/dfs/translated.es']

In [35]:
def evaluate(path_obj, language, name, spacy_model_from_disk=False, only_specific_models=False, number_of_batch=None, start_batch=False, end_batch=False):
  batches = [] 
  if only_specific_models:
    batches = only_load_specific_batches(path_obj, start_batch=False, num_batch=number_of_batch, end_batch=False)
  else: 
    # load data
    data = load_data(path_obj)
    # batch data
    batches = create_batches(data, 1000)
  # run spacy tokenizer and classifier
  dfs = create_token_dataframes(batches, language, name)
  # pack all into one df
  df = concatenate_df_batches(dfs)
  # filter stop and non alphanumericals
  clean_df = filter_stop_non_alpha(df)
  try:
    # save entire df to disk
    superclean_df = clean_df.dropna(how='any',axis=0)  
    dfs_path = path.join("/content/drive/MyDrive/mlt_final_project", "dfs")
    superclean_df.to_csv(path.join(dfs_path, name))
  except EnvironmentError as e:
    print(e.with_traceback())
  # simpsons diversity
  sim_div = simpson_diversity(clean_df)
  # shannon entropy
  shan_entr = shannon_entropy(clean_df)
  # lexical frequency
  lexical_freq = get_lfp(clean_df)
  # legoma count -> hapax legomenon, bi legomenon tri legomenon
  legomas = count_legomas(clean_df)
  # lemma count -> filtered stop words and non alphanumericals
  lemma_dict = {}
  struct_dict = {}
  dep_dict = {}
  for _, row in clean_df.iterrows():
    if row['Lemma'] not in lemma_dict.keys():
      lemma_dict.update({row['Lemma']: 0})
    else:
      lemma_dict[row['Lemma']] += 1
  # noun, verb, adjective, adverb, sub. conjunction count
    if row['Pos'] not in struct_dict.keys():
      struct_dict.update({row['Pos']: 0})
    else:
      struct_dict[row['Pos']] += 1
  # dependency count
    if row['Dep'] not in dep_dict.keys():
      dep_dict.update({row['Dep']: 0})
    else:
      dep_dict[row['Dep']] += 1

  return sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict

#    
def load_and_evaluate_df():
  # metrics_list = []
  # dfs_path = path.join("/content/drive/MyDrive/mlt_final_project", "dfs")
  paths = ['/content/drive/MyDrive/mlt_final_project/dfs/pp_dees.es', 
           '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_1', 
           '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_0',
           '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_2',
           '/content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_3',
           '/content/drive/MyDrive/mlt_final_project/dfs/translated.es']
  norm_const = None
  index_dict = None
  for f_path in paths:
    print(f'Loading {f_path}')
    output_path = f_path.replace(".csv", "").split("/")[-1]
    df = pd.read_csv(f_path, header=0)
    clean_df = df.dropna(how='any',axis=0)
    print(f'Loaded {f_path}')
    print(clean_df.loc[0,:])
    print(f'DataFrame has Null values: {clean_df.isnull().values.any()}')
    # type token ratio
    ttr_words = ttr_ratio(clean_df, lemma=False, sort_punctuation_out=True, sort_stopwords_out=False)
    # type token ratio lemmas
    ttr_lemmas = ttr_ratio(clean_df, lemma=True, sort_punctuation_out=True, sort_stopwords_out=False)
    # simpson diversity
    sim_div = simpson_diversity(clean_df)
    if "pp_deen" in f_path:
      # shannon entropy
      shan_entr, norm_const_ = shannon_entropy(clean_df, normalizing_constant=None)  # (dataframe, normalizing_constant=None)
      norm_const = norm_const_
    else:
      shan_entr = shannon_entropy(clean_df, normalizing_constant=norm_const)  # (dataframe, normalizing_constant=None) 
    if "pp_deen" in f_path:
      # lexical frequency
      lexical_freq = get_lfp(clean_df, index_dict=None) # (data_frame, index_dict=None, return_index_dict=False, sort_punctuation_out=True, sort_stopwords_out=False)
      index_dict = lexical_freq[1]
    else:
      # lexical frequency
      lexical_freq = get_lfp(clean_df, index_dict=index_dict) # (data_frame, index_dict=None, return_index_dict=False, sort_punctuation_out=True, sort_stopwords_out=False)
    # legoma count -> hapax legomenon, bi legomenon tri legomenon
    legomas = count_legomas(clean_df)
    # lemma count -> filtered stop words and non alphanumericals
    lemma_dict = {}
    struct_dict = {}
    dep_dict = {}
    for _, row in clean_df.iterrows():
      if row['Lemma'] not in lemma_dict.keys():
        lemma_dict.update({row['Lemma']: 0})
      else:
        lemma_dict[row['Lemma']] += 1
    # noun, verb, adjective, adverb, sub. conjunction count
      if row['Pos'] not in struct_dict.keys():
        struct_dict.update({row['Pos']: 0})
      else:
        struct_dict[row['Pos']] += 1
    # dependency count
      if row['Dep'] not in dep_dict.keys():
        dep_dict.update({row['Dep']: 0})
      else:
        dep_dict[row['Dep']] += 1
    process_created_dfs(output_path, ttr_words, ttr_lemmas, sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict)



In [20]:
# load saved spacy models from disk to continue evaluation
def load_saved_spacy_model_to_df(language):
  spacy_models_path = path.join("/content/drive/MyDrive/mlt_final_project", "spacy_models")
  first_it_model_paths = glob.glob(f'{spacy_models_path}/{language}_old/{language}_spacy_model_forward_trl_de{language}_0_*')
  sec_it_model_paths = glob.glob(f'{spacy_models_path}/{language}_old/{language}_spacy_model_forward_trl_de{language}_1_*')
  third_it_model_paths = glob.glob(f'{spacy_models_path}/{language}_old/{language}_spacy_model_forward_trl_de{language}_2_*')
  fourth_it_model_paths = glob.glob(f'{spacy_models_path}/{language}_old/{language}_spacy_model_forward_trl_de{language}_3_*')
  first_it_models = []
  sec_it_models = []
  third_it_models = []
  fourth_it_models = []
  if "es" in language:
    first_it_models = [nlp_es.from_disk(model) for model in first_it_model_paths]
    sec_it_models = [nlp_es.from_disk(model) for model in sec_it_model_paths]
    third_it_models = [nlp_es.from_disk(model) for model in third_it_model_paths]
    fourth_it_models = [nlp_es.from_disk(model) for model in fourth_it_model_paths]
  else:
    first_it_models = [nlp_en.from_disk(model) for model in first_it_model_paths]
    sec_it_models = [nlp_en.from_disk(model) for model in sec_it_model_paths]
    third_it_models = [nlp_en.from_disk(model) for model in third_it_model_paths]
    fourth_it_models = [nlp_en.from_disk(model) for model in fourth_it_model_paths]

  

In [15]:
def only_load_specific_batches(path_obj, start_batch=False, num_batch=0, end_batch=True):
  # load data
  data = load_data(path_obj)
  # batch data
  batches = create_batches(data, 1000)
  if start_batch:
    return batches[num_batch:]
  elif end_batch:
    return batches[num_batch-1:num_batch]
  else:
    return batches

In [32]:
# writes the given parameters (evaluation metrics) to disk as .csv and .txt files
def write_metrics_to_disk(ttr_words, ttr_lemmas, sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict, file_path):
  #sum_structs = sum([value for key, value in struct_dict.items()])
  if os.path.exists(f'{file_path}.txt'):
    append_write = 'a' # append if already exists
  else:
    append_write = 'w' # make a new file if not
  if os.path.exists(f'{file_path}.csv'):
    append_write_csv = 'a' # append if already exists
  else:
    append_write_csv = 'w' # make a new file if not
  try:
    with open(f'{file_path}.txt', append_write) as f:  
      f.write(f'- Grammatical structure count -\n')
      #f.write(f'Sum of all structs is {sum_structs}\n')
      #for key, value in struct_dict.items():
      #  f.write(f'{key}: {value}\n')
      f.write(f'Nouns: {struct_dict["NOUN"]}, Verbs: {struct_dict["VERB"]}, Adverbs: {struct_dict["ADV"]}, Adjectives: {struct_dict["ADJ"]}\n')
      f.write(f'- Other metrics -\n')
      f.write(f'TTR words is: {ttr_words}\n')
      f.write(f'TTR lemmas is: {ttr_lemmas}\n')
      f.write(f'The shannon entropy is: {shan_entr}\n')
      f.write(f'The simpson diversity is: {sim_div}\n')
      f.write(f'The lexical frequency is: First thousand {lexical_freq[0][0]}, second thousand {lexical_freq[0][1]}, rest {lexical_freq[0][2]}\n')
      f.write(f'The hapax legoma are: {legomas[0]}\n')
      f.write(f'The bi legoma are: {legomas[1]}\n')
      f.write(f'The tri legoma are: {legomas[2]}\n')
      f.write(f'The number of lemmas are: {len(lemma_dict)}\n')
      f.write(f'- Syntactical dependencies -\n')
      for key, value in dep_dict.items():
        f.write(f'{key}: {value}\n')
  except EnvironmentError as e:
    print(f'Error occured while writing {file_path}.txt')
    print(e.with_traceback())

  try:
    with open(f'{file_path}.csv', append_write_csv) as f:
      f.write(f'Nouns, {struct_dict["NOUN"]}\n')
      f.write(f'Verbs, {struct_dict["VERB"]}\n')
      f.write(f'Adverbs, {struct_dict["ADV"]}\n')
      f.write(f'Adjectives, {struct_dict["ADJ"]}\n')
      # f.write(f'Subordinating_conjunctions, {struct_dict["SCONJ"]}\n')
      f.write(f'TTR_words, {ttr_words}\n')
      f.write(f'TTR_lemmas, {ttr_lemmas}\n')
      f.write(f'shannon_entropy, {shan_entr}\n')
      f.write(f'simpson_diversity, {sim_div}\n')
      f.write(f'lexical_frequency_first_thousand, {lexical_freq[0][0]}\n')
      f.write(f'lexical_frequency_second_thousand, {lexical_freq[0][1]}\n')
      f.write(f'lexical_frequency_third_thousand, {lexical_freq[0][2]}\n')
      f.write(f'hapax_legoma, {legomas[0]}\n')
      f.write(f'bi_legoma, {legomas[1]}\n')
      f.write(f'tri_legoma, {legomas[2]}\n')
      f.write(f'lemmas, {len(lemma_dict)}\n')
      for key, value in dep_dict.items():
        f.write(f'{key}, {value}\n')
  except EnvironmentError as e:
    print(f'Error occured while writing {file_path}.csv')
    print(e.with_traceback())

In [33]:
def process():
  it_trl_path = "/content/drive/MyDrive/mlt_final_project/iterative_translation"
  translated_path = "/content/drive/MyDrive/mlt_final_project/translated"
  preprocessed_path = "/content/drive/MyDrive/mlt_final_project/preprocessed"
  # translation files from german to english
  it_trl_deen_paths = [path.join(it_trl_path, f'forward_trl_deen_{num}.txt') for num in range(4)]
  # translation files from german to spanish
  it_trl_dees_paths = [path.join(it_trl_path, f'forward_trl_dees_{num}.txt') for num in range(4)]
  translated_en_path = path.join(translated_path, "de-en", "translated.en")
  translated_es_path = path.join(translated_path, "de-es", "translated.es")
  reference_en_path = path.join(preprocessed_path, "de-en", "pp_deen.en")
  reference_es_path = path.join(preprocessed_path, "de-es", "pp_dees.es")
  eval_path = path.join("/content/drive/MyDrive/mlt_final_project", "evaluation_two")

  # path.join(it_trl_path, f'forward_trl_deen_1.txt') von batch 19 bis 50
  # path.join(it_trl_path, f'forward_trl_deen_2.txt') ganz
  # path.join(it_trl_path, f'forward_trl_deen_3.txt') ganz
  # f'forward_trl_dees_3.txt', f'forward_trl_deen_2.txt', f'forward_trl_deen_3.txt'
  all_files = [path.join(preprocessed_path, "de-es", "pp_dees.es"), path.join(it_trl_path, f'forward_trl_dees_1.txt'), path.join(it_trl_path, f'forward_trl_dees_0.txt'), path.join(it_trl_path, f'forward_trl_dees_2.txt'),
               path.join(it_trl_path, f'forward_trl_dees_3.txt'), translated_es_path]
  for path_obj in all_files:
    output_path = path_obj.replace(".txt", "").split("/")[-1]
    # if "es" in path_obj and "pp_deen" not in path_obj:
    sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict = evaluate(path_obj, "es", output_path, only_specific_models=False, number_of_batch=None, start_batch=False, end_batch=False)
    # else:
    # sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict = evaluate(path_obj, "en", output_path, only_specific_models=False, number_of_batch=None, start_batch=False, end_batch=False)

  total_output_path = path.join(eval_path, output_path)
  # save all evaluation metrics
  write_metrics_to_disk(sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict, total_output_path)


def process_created_dfs(output_path, ttr_words, ttr_lemmas, sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict):
  eval_two_path = path.join("/content/drive/MyDrive/mlt_final_project", "evaluation_two")
  print(f'Name of file in write method is {output_path}')
  output_tot_path = path.join(eval_two_path, output_path)
  print(f'Final output path (write method) is {output_tot_path}')
  write_metrics_to_disk(ttr_words, ttr_lemmas, sim_div, shan_entr, lexical_freq, legomas, lemma_dict, struct_dict, dep_dict, output_tot_path) 

In [36]:
load_and_evaluate_df()

Loading /content/drive/MyDrive/mlt_final_project/dfs/pp_dees.es
Loaded /content/drive/MyDrive/mlt_final_project/dfs/pp_dees.es
Unnamed: 0              0
Text          Reanudación
Lemma         Reanudación
Pos                 PROPN
Tag                 PROPN
Dep                  ROOT
Shape               Xxxxx
Name: 0, dtype: object
DataFrame has Null values: False
Name of file in write method is pp_dees.es
Final output path (write method) is /content/drive/MyDrive/mlt_final_project/evaluation_two/pp_dees.es
Loading /content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_1
Loaded /content/drive/MyDrive/mlt_final_project/dfs/forward_trl_dees_1
Unnamed: 0              0
Text          Reanudación
Lemma         Reanudación
Pos                 PROPN
Tag                 PROPN
Dep                  ROOT
Shape               Xxxxx
Name: 0, dtype: object
DataFrame has Null values: False
Name of file in write method is forward_trl_dees_1
Final output path (write method) is /content/drive/MyDriv

In [ ]:
process() # (path_obj, language, name,